# Install some basic library:

In [1]:
!pip install -qU langchain_community pypdf
!pip install langchain-groq
!pip install sentence_transformers
!pip install faiss-gpu
!pip install langgraph
!pip install --upgrade pydantic

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.8.2 requires cubinlinker, which is not installed.
cudf 24.8.2 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.8.2 requires ptxcompiler, which is not installed.
cuml 24.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 24.8.2 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.8.2 requires cuda-python<12.0a0,>=11.7.1, but you have cuda-python 12.6.0 which is incompatible.
distributed 2024.7.1 requires dask==2024.7.1, but you have dask 2024.8.1 which is incompatible.
google-cloud-bigquery 2.34.4 requires packaging<22.0dev,>=14.3, but you have packaging 24.1 which is incompatible.
jupyterlab 4.2.4 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.1.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which 

# Now Put Your API key:

In [2]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_PROJECT'] = 'advanced-rag'
os.environ['LANGCHAIN_API_KEY'] = "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"
os.environ['GROQ_API_KEY'] = "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"
os.environ['TAVILY_API_KEY'] = "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"

# Import Libraries:

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_groq import ChatGroq
from langchain_community.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_core.output_parsers import StrOutputParser
from langchain.load import dumps, loads
from typing_extensions import TypedDict
from operator import itemgetter
from typing import List
from langchain_core.runnables import RunnablePassthrough
import warnings
warnings.filterwarnings("ignore")

# Load the documents:

In [4]:
loader_1 = PyPDFLoader('/kaggle/input/kitchen-dataset/cook_book.pdf')
cook_pdf =loader_1.load()

loader_2 = PyPDFLoader('/kaggle/input/kitchen-dataset/List_of_all_recipe.pdf')
cook_pdf.extend(loader_2.load())

In [5]:
cook_pdf

[Document(metadata={'source': '/kaggle/input/kitchen-dataset/cook_book.pdf', 'page': 0}, page_content='1. Nargisy Kababs: \n Ingredients \n\uf0b7 1 kg minced meat \n\uf0b7 2 medium onions, cut into large pieces \n\uf0b7 1/2 cup curd \n\uf0b7 1 tbsp ginger-garlic paste \n\uf0b7 1 tsp red chilli powder \n\uf0b7 1 tsp garam masala \n\uf0b7 1/4 tsp turmeric powder \n\uf0b7 1 tbsp soy sauce \n\uf0b7 salt as per taste \n\uf0b7 1 tbsp vinegar \n\uf0b7 1/4 cup \nFor Filling: \n\uf0b7 4 boiled eggs, finely chopped \n\uf0b7 1 cup coriander leaves, \n\uf0b7 finely chopped \n\uf0b7 1 cup mint leaves, finely chopped \n\uf0b7 2 or 3 green chilies finely chopped \n\uf0b7 1 tsp. chat masala \n For Frying: \n\uf0b7  1/4 cup besan \n\uf0b7  2 eggs \n\uf0b7  1/4 tsp salt \n\uf0b7  1/4 tsp turmeric powder \n \nMethod \nCook onions in oil until light brown. Add yogurt, garlic & ginger paste, salt, red chili, \ngaram masala, Soya sauce & vinegar to the onions and cook for 6 to 8 minutes. \n Add mince meat t

# Split the Documents:

In [6]:
text_spliter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=150)
splits = text_spliter.split_documents(cook_pdf)

In [7]:
splits

[Document(metadata={'source': '/kaggle/input/kitchen-dataset/cook_book.pdf', 'page': 0}, page_content='1. Nargisy Kababs: \n Ingredients \n\uf0b7 1 kg minced meat \n\uf0b7 2 medium onions, cut into large pieces \n\uf0b7 1/2 cup curd \n\uf0b7 1 tbsp ginger-garlic paste \n\uf0b7 1 tsp red chilli powder \n\uf0b7 1 tsp garam masala \n\uf0b7 1/4 tsp turmeric powder \n\uf0b7 1 tbsp soy sauce \n\uf0b7 salt as per taste \n\uf0b7 1 tbsp vinegar \n\uf0b7 1/4 cup \nFor Filling: \n\uf0b7 4 boiled eggs, finely chopped \n\uf0b7 1 cup coriander leaves, \n\uf0b7 finely chopped \n\uf0b7 1 cup mint leaves, finely chopped \n\uf0b7 2 or 3 green chilies finely chopped \n\uf0b7 1 tsp. chat masala \n For Frying:'),
 Document(metadata={'source': '/kaggle/input/kitchen-dataset/cook_book.pdf', 'page': 0}, page_content='\uf0b7 finely chopped \n\uf0b7 1 cup mint leaves, finely chopped \n\uf0b7 2 or 3 green chilies finely chopped \n\uf0b7 1 tsp. chat masala \n For Frying: \n\uf0b7  1/4 cup besan \n\uf0b7  2 eggs \

# Embedding the documents:

In [8]:
model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
hf_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)
vectorstore = FAISS.from_documents(documents=splits, 
                                    embedding=hf_embeddings)

retriever = vectorstore.as_retriever()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# LLMs

## Generate multiple query prompt: 

In [9]:
# Multi Query: Different Perspectives
template = """You are an AI language model assistant. Your task is to generate seven 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines. Original question: {question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)

generate_queries = (
    prompt_perspectives 
    | ChatGroq(temperature=0) 
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

In [10]:
generate_queries.invoke("how to cook chicken?")

['1. What are the steps to prepare a chicken meal?',
 '2. Can you provide a guide on cooking chicken for beginners?',
 '3. What are some delicious chicken recipes for dinner?',
 "4. How do I cook chicken properly to ensure it's safe to eat?",
 '5. What are the best methods for cooking chicken: grilling, baking, or frying?',
 '6. I need help with a chicken dish for a special occasion, can you provide some ideas?',
 '7. Can you suggest some easy and healthy chicken cooking techniques?']

In [11]:
def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Get unique documents
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]

# Retrieve
question = "how to cook chicken?"
retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question":question})
len(docs)

14

## Retrival gradder:

In [13]:
# I used the class for data modeling 
class GradeRetrival(BaseModel):
    """Binary score for relevance check on retrieved documents."""

    binary_score: str = Field(description="Documents are relevant to the question, 'yes' or 'no'")

llm = ChatGroq(temperature=0)
structured_llm_grader = llm.with_structured_output(GradeRetrival)

# Prompt 
system = """You are a grader assessing relevance of a retrieved document to a user question. \n 
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""
grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)

retrieval_grader = grade_prompt | structured_llm_grader
question = "What are the steps to prepare a chicken meal?"
docs = retriever.get_relevant_documents(question)
doc_txt = docs
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

binary_score='yes'


## Generate answer:

In [14]:
# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}

Please provide the answer in the following format:
1. List of all ingredients required.
2. Step-by-step method for cooking.
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatGroq(temperature=0)

final_rag_chain = ({"context": retrieval_chain, "question": itemgetter("question")} | prompt| llm| StrOutputParser()
)
# 
ans_generation = final_rag_chain.invoke({"question":question})
print(ans_generation)

1. List of all ingredients required:
   - Chicken (1 large bowl cut into pieces)
   - Yogurt (1 cup)
   - Ginger paste (1 tbsp)
   - Garlic paste (1 tbsp)
   - White vinegar (4 tbsp)
   - Lemon juice (3 tbsp)
   - Red chilli powder (2 tbsp)
   - Olive oil (4 tbsp)
   - Salt and black pepper to taste
   - Onions (2 large, finely chopped)
   - Tomatoes (3-4, chopped)
   - Green chillies (3-4)
   - Garam masala (1/2 tsp)
   - Coriander leaves (1/2 tsp)
   - Chinese salt (1/2 tsp)
   - Turmeric powder (1/4 tsp)
   - Red chilli powder (1 tsp)
   - Ketchup (1 tbsp)
   - Yogurt (1/2 cup)
   - Butter (2 tbsp)
   - Black pepper (1 tsp)
   - Salt (1 tsp)
   - Eggs (for dipping)
   - Oil (for frying)

2. Step-by-step method for cooking:
   1. Skin and bone the chicken. Cut it into 4 parts. Prick it well with a fork or sharp knife.
   2. Put the yogurt in a large bowl, add ginger, garlic, white vinegar, lemon juice, chilli powder, pepper, and salt. Mix well.
   3. Add chicken to the marinade, turn

## fallback prompt: 

In [16]:
fallback_prompt = ChatPromptTemplate.from_template(
    (
        "You are a friendly medical assistant created by JUNAID BADSHAH.\n"
        "Do not respond to queries that are not related to kitchen_recipe.\n"
        "If a query is not related to kitchen_recipe, acknowledge your limitations.\n"
        "Provide concise responses to only kitchen_recipe-related queries.\n\n"
        "Current conversations:\n\n{chat_history}\n\n"
        "human: {query}"
    )
)

fallback_chain = (
    {
        "chat_history": lambda x: "\n".join(
            [
                (
                    f"human: {msg.content}"
                    if isinstance(msg, HumanMessage)
                    else f"AI: {msg.content}"
                )
                for msg in x["chat_history"]
            ]
        ),
        "query": itemgetter("query") ,
    }
    | fallback_prompt
    | llm
    | StrOutputParser()
)

fallback_chain.invoke(
    {
        "query": "Hello",
        "chat_history": [],
    }
)

"Hello! I'm here to help with kitchen recipes. What can I assist you with today?"

## Hallucination Grader:

In [17]:
# I used the class for data modeling 
class HallucinationGrader(BaseModel):
    """Binary score for hallucination present in generation answer."""
    binary_score: str = Field(description= "Answer is grounded in the fact, 'yes' or 'No'")

# so here I use ChatGroq LLm
llm = ChatGroq(temperature=0)
structure_llm_grader = llm.with_structured_output(HallucinationGrader)

# Prompt
system = """You are a grader assessing whether an LLM generation is grounded in / supported by a set of retrieved facts. \n
     Give a binary score 'yes' or 'no'. 'Yes' means that the answer is grounded in / supported by the set of facts."""
hallucination_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Set of facts: \n\n {documents} \n\n LLM generation: {generation}"),
    ]
)


hallucination_grader = hallucination_prompt | structured_llm_grader
hallucination_grader.invoke({"documents": docs, "generation": ans_generation})

GradeRetrival(binary_score='no')

## Answer grader

In [18]:
class GradeAnswer(BaseModel):
    """Binary score to assess answer addresses question."""

    binary_score: str = Field(description="Answer addresses the question, 'yes' or 'no'")
        
# LLM with function call
llm = ChatGroq(temperature=0)
structured_llm_grader = llm.with_structured_output(GradeAnswer)

# Prompt
system = """You are a grader assessing whether an answer addresses / resolves a question \n
     Give a binary score 'yes' or 'no'. Yes' means that the answer resolves the question."""
answer_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "User question: \n\n {question} \n\n LLM generation: {generation}"),
    ]
)

answer_grader = answer_prompt | structured_llm_grader
answer_grader.invoke({"question": question,"generation": ans_generation})

GradeAnswer(binary_score='yes')

In [19]:
# LLM
llm = ChatGroq(temperature=0)

# Prompt
system = """You a question re-writer that converts an input question to a better version that is optimized \n
     for vectorstore retrieval. Look at the input and try to reason about the underlying sematic intent / meaning. Just say the question. I dont need any explanation just question is enough"""
re_write_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Here is the initial question: \n\n {question} \n Formulate an improved question."),
    ]
)

question_rewriter = re_write_prompt | llm | StrOutputParser()
question_rewriter.invoke({"question": question})

'"How to cook a chicken meal: list the preparation steps"'

# Graph state

In [20]:
class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        documents: list of documents
    """
    question : str
    ans_generation : str
    documents : List[str]

# Node:

In [21]:
def retrieve(state):
    """
    Retrieve documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    question = state["question"]

    # Retrieval
    documents = retriever.get_relevant_documents(question)
    return {"documents": documents, "question": question}

def generate(state):
    """
    Generate answer

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]

    # RAG generation
    ans_generation = final_rag_chain.invoke({"context": documents, "question": question})
    return {"documents": documents, "question": question, "generation": ans_generation}


def fallback_node(state: dict):
    """
    Fallback to this node when there is no tool call
    """
    query = state["query"]
    chat_history = state["chat_history"]
    ans_generation = fallback_chain.invoke({"query": query, "chat_history": chat_history})
    return {"generation": ans_generation}


def grade_documents(state):
    """
    Determines whether the retrieved documents are relevant to the question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with only filtered relevant documents
    """
    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state["documents"]

    # Score each doc
    filtered_docs = []
    for d in documents:
        score = retrieval_grader.invoke({"question": question, "document": d.page_content})
        grade = score.binary_score
        if grade == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            continue
    return {"documents": filtered_docs, "question": question}

def transform_query(state):
    """
    Transform the query to produce a better question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates question key with a re-phrased question
    """

    print("---TRANSFORM QUERY---")
    question = state["question"]
    documents = state["documents"]

    # Re-write question
    better_question = question_rewriter.invoke({"question": question})
    return {"documents": documents, "question": better_question}

# Edges:

In [22]:
def decide_to_generate(state):
    """
    Determines whether to generate an answer, or re-generate a question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("---ASSESS GRADED DOCUMENTS---")
    question = state["question"]
    filtered_documents = state["documents"]

    if not filtered_documents:
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print("---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, TRANSFORM QUERY---")
        return "transform_query"
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: GENERATE---")
        return "generate"

def grade_generation_v_documents_and_question(state):
    """
    Determines whether the generation is grounded in the document and answers question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Decision for next node to call
    """

    print("---CHECK HALLUCINATIONS---")
    question = state["question"]
    documents = state["documents"]
    ans_generation = state["generation"]

    score = hallucination_grader.invoke({"documents": documents, "generation": ans_generation})
    grade = score.binary_score

    # Check hallucination
    if grade == "yes":
        print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")
        # Check question-answering
        print("---GRADE GENERATION vs QUESTION---")
        score = answer_grader.invoke({"question": question,"generation": ans_generation})
        grade = score.binary_score
        if grade == "yes":
            print("---DECISION: GENERATION ADDRESSES QUESTION---")
            return "useful"
        else:
            print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
            return "not useful"
    else:
        print("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
        return "not supported"

In [23]:
from langgraph.graph import END, StateGraph

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("retrieve", retrieve)
workflow.add_node("grade_documents", grade_documents)
workflow.add_node("fallback_node", fallback_node)
workflow.add_node("generate", generate) 
workflow.add_node("transform_query", transform_query)

# build graph
workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "transform_query": "transform_query",
        "generate": "generate",
        "fallback": "fallback_node",
    },
)
workflow.add_edge("transform_query", "retrieve")
workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "not supported": "generate",
        "useful": END,
        "not useful": "transform_query",
    },
)
workflow.add_edge("fallback_node", END)

# Compile
app = workflow.compile()

In [25]:
from pprint import pprint

# Run
inputs = {"question": "What are the steps to prepare a chicken meal?"}

for output in app.stream(inputs):
    for key, value in output.items():
        # Node
        pprint(f"Node '{key}':")
        # Optional: print full state at each node
        # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
    pprint("\n---\n")

# Final generation
pprint(value["generation"])

---RETRIEVE---
"Node 'retrieve':"
'\n---\n'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---


BadRequestError: Error code: 400 - {'error': {'message': "Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': '<tool-use>\n{"tool\\_call": {\n"id": "pending",\n"type": "function",\n"function": {\n"name": "GradeRetrival"\n},\n"parameters": {\n"binary\\_score": "yes"\n}\n}}\n</tool-use>'}}